## In this notebook, we plot various candle prices, and the moving average

In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instruments

In [3]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16, 64]
i_pair = instruments.Instrument.get_instrument_by_name(pair)

df = pd.read_pickle(utils.get_hist_data_filename(pair, granularity))
#df.head()
#df.describe()
#df.info()
#df.columns

In [4]:
# convert object(strings) types to integer types for data manipulation
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]

df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [6]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
    
df_ma.dropna(inplace=True)

# reset index
df_ma.reset_index(drop=True, inplace=True)

In [7]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2024-06-17T02:00:00.000000000Z,1.07016,1.07049,1.06990,1.07042,1.069810,1.075689
1,2024-06-17T03:00:00.000000000Z,1.07043,1.07048,1.07019,1.07038,1.069871,1.075595
2,2024-06-17T04:00:00.000000000Z,1.07038,1.07055,1.07021,1.07042,1.069965,1.075403
3,2024-06-17T05:00:00.000000000Z,1.07043,1.07053,1.06992,1.06998,1.070155,1.075190
4,2024-06-17T06:00:00.000000000Z,1.06996,1.07040,1.06864,1.06976,1.070220,1.074955


In [8]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [9]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [10]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [11]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [12]:
df_trades = df_ma[df_ma.IS_TRADE != 0].copy()

In [13]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
22,2024-06-18T00:00:00.000000000Z,1.07402,1.07408,1.07356,1.07360,1.072495,1.072378,0.000117,-0.000173,1
81,2024-06-20T11:00:00.000000000Z,1.07288,1.07322,1.07130,1.07150,1.073616,1.073628,-0.000012,0.000148,-1
132,2024-06-24T14:00:00.000000000Z,1.07426,1.07465,1.07272,1.07301,1.071071,1.071047,0.000024,-0.000226,1
166,2024-06-26T00:00:00.000000000Z,1.07091,1.07126,1.07062,1.07115,1.071136,1.071292,-0.000156,0.000019,-1
209,2024-06-27T19:00:00.000000000Z,1.07060,1.07074,1.07014,1.07030,1.070226,1.070168,0.000058,-0.000074,1


In [ ]:
# assess performance of strategy

In [85]:
# plot last 100 candles
df_plot = df_ma.iloc[:37].copy()

In [86]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, 
    open=df_plot.mid_o, 
    high=df_plot.mid_h, 
    low=df_plot.mid_l, 
    close=df_plot.mid_c,
    line=dict(width=1),
    opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
))
fig.update_layout(
    width=1000, height=400,
    font=dict(size=10, color="#e1e1e1"),
    margin=dict(l=10, r=10, b=10, t=10),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e"
)
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True, fixedrange=True, #rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()